## 커스텀 손실함수 정의하기

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

# mse
def mse(y_true, y_pred):
    return tf.reduce_mean(tf.math.square(y_true - y_pred), axis = -1)

# binary_crossentropy
def binary_crossentropy(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, K.epsilon(), 1.0 - K.epsilon())
    total_loss = -y_true * tf.math.log(y_pred) - (1.0 - y_true) * tf.math.log(1.0 - y_pred)
    
    return total_loss

# categorical_crossentropy
def categorical_crossentropy(y_true, y_pred):
    y_pred = y_pred / tf.reduce_sum(y_pred, axis = -1, keepdims = True)
    y_pred = tf.clip_by_value(y_pred, K.epsilon(), 1.0 - K.epsilon())
    
    return -tf.reduce_sum(y_true * tf.math.log(y_pred), axis = -1)

## 커스텀 손실함수 - MNIST 학습

In [ ]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data(path='mnist.npz')

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                  test_size = 0.3, random_state = 777)

x_train = (x_train.reshape(-1, 28, 28, 1)) / 255
x_val = (x_val.reshape(-1, 28, 28, 1)) / 255
x_test = (x_test.reshape(-1, 28, 28, 1)) / 255

from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.layers import Input

inputs = Input(shape = (28, 28, 1))

x = Conv2D(32, (3, 3), activation = 'relu')(inputs)
x = Conv2D(32, (3, 3), activation = 'relu')(x)
x = MaxPooling2D(strides = 2)(x)

x = GlobalAveragePooling2D()(x)
x = Dense(10, activation = 'softmax')(x)

model = Model(inputs = inputs, outputs = x)

# 정의한 손실 함수를 사용합니다.
model.compile(optimizer = 'adam', 
              loss = categorical_crossentropy,
              metrics = ['acc'])

model.fit(x_train, y_train,
         batch_size = 32,
         validation_data = (x_val, y_val),
         epochs = 10)